In [69]:
import argparse
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from collections import Counter
from multimodal.multimodal_data_module import MultiModalSAYCamDataModule, read_vocab
from multimodal.multimodal import TextEncoder, MultiModalModel
from multimodal.lm_lit import LMLitModel

from train_lm import _setup_parser

In [132]:
parser = _setup_parser()
args = parser.parse_args(args="".split())

data = MultiModalSAYCamDataModule(args)
data.prepare_data()
data.setup()
vocab = read_vocab()
vocab_size = len(vocab)
print(f'vocab_size = {vocab_size}')
idx2word = {idx: word for word, idx in vocab.items()}
for idx in range(vocab_size):
    assert idx in idx2word
args.input_dim = vocab_size
text_encoder = TextEncoder(args=args)
lit_model = LMLitModel(text_encoder, args)
lit_model = lit_model.load_from_checkpoint("checkpoints/lm_embedding_type_spatial_text_encoder_lstm_dropout_0.5_batch_size_16/last.ckpt")
lit_model.cuda()
lit_model.eval()

Calling prepare_data!
SAYCam transcripts have already been downloaded. Skipping this step.
Transcripts have already been renamed. Skipping this step.
Transcripts have already been preprocessed. Skipping this step.
Training frames have already been extracted. Skipping this step.
Training metadata files have already been created . Skipping this step.
Evaluation frames have already been extracted. Skipping this step.
Evaluation metadata files have already been created . Skipping this step.
Vocabulary file already exists. Skipping this step.
Calling setup!
vocab_size = 5589


LMLitModel(
  (text_encoder): TextEncoder(
    (embedding): Embedding(5589, 128, padding_idx=0)
    (lstm): LSTM(128, 128, dropout=0.5)
  )
  (output_layer): Linear(in_features=128, out_features=5589, bias=True)
)

In [133]:
from tqdm import tqdm

sum_vectors = torch.zeros(vocab_size, lit_model.text_encoder.hidden_dim, device='cuda')
cnt_vectors = torch.zeros(vocab_size, dtype=torch.int, device='cuda')

for x, y, y_len in tqdm(data.train_dataloader()):
    y, y_len = y.cuda(), y_len.cuda()
    outputs = lit_model.text_encoder(y, y_len)
    for i in range(len(y)):
        for j in range(y_len[i].item()):
            token_id = y[i, j]
            vector = outputs[i, j].detach()
            cnt_vectors[token_id] += 1
            sum_vectors[token_id] += vector

mean_vectors = sum_vectors / cnt_vectors.unsqueeze(-1)
vectors = mean_vectors.cpu().numpy()
for i in range(len(cnt_vectors)):
    if cnt_vectors[i].item() == 0:
        vectors[i].fill(0.)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 8289/8289 [01:22<00:00, 100.19it/s]


In [134]:
tokens = input().split()
token_ids = []
print('not found:', end='')
for s in tokens:
    try:
        token_ids.append(lit_model.word2idx[s])
    except KeyError:
        print(f' {s}')
print()
np.set_printoptions(precision=2)
V = vectors[token_ids]
V = V / np.linalg.norm(V, axis=1, keepdims=True)
print(V @ V.T)

mom mommy daddy go play
not found:
[[1.   0.39 0.22 0.09 0.13]
 [0.39 1.   0.35 0.1  0.18]
 [0.22 0.35 1.   0.11 0.1 ]
 [0.09 0.1  0.11 1.   0.27]
 [0.13 0.18 0.1  0.27 1.  ]]


In [137]:
from scipy.cluster.hierarchy import dendrogram, linkage

selected = cnt_vectors.cpu().numpy() >= 6
selected_vectors = vectors[selected]
selected_indices = np.arange(len(vectors))[selected]
n_samples = len(selected_vectors)

Z = linkage(selected_vectors, method='ward')

print(Z.shape)

(1535, 4)


In [140]:
def llf(idx):
    if idx < n_samples:
        idx = selected_indices[idx]
        return lit_model.idx2word[idx] + f'   {cnt_vectors[idx].item()}'
    else:
        return f'{idx - n_samples} {int(Z[idx - n_samples, 3])} {Z[idx - n_samples, 2]:.3f}'

p = 2000

plt.figure(figsize=(25, 0.4 * p))
_ = dendrogram(
    Z,
    #truncate_mode='lastp',
    #p=p,
    orientation='left',
    leaf_rotation=0.,
    leaf_font_size=16.,
    leaf_label_func=llf,
)